## Check crashes in small clusters

Get crashes in smaller clusters.

In [2]:
import pandas as pd
import utils.config as config
import utils.util as util
import numpy as np

df_err_grouped_k = pd.read_excel(config.path_default.joinpath('Clustering/clusters_Kaggle.xlsx'))
df_err_grouped_g = pd.read_excel(config.path_default.joinpath('Clustering/clusters_GitHub.xlsx'))

In [3]:
selected_clusters_g = pd.read_excel(config.path_default.joinpath('Sampling/cluster_size_samples_g.xlsx'))
selected_clusters_k = pd.read_excel(config.path_default.joinpath('Sampling/cluster_size_samples_k.xlsx'))

# big clusters (sample size >= 10)
clusters_big_k = selected_clusters_k[selected_clusters_k.sample_size>=10].cluster_id[1:]
clusters_big_g = selected_clusters_g[selected_clusters_g.sample_size>=10].cluster_id[1:]

In [25]:
# get crashes from small clusters (sample size < 10)
df_err_grouped_k_small = df_err_grouped_k[~df_err_grouped_k.pregroup_cluster.isin(clusters_big_k)]
df_err_grouped_g_small = df_err_grouped_g[~df_err_grouped_g.pregroup_cluster.isin(clusters_big_g)]

In [ ]:
# randomly select 10 small clusters from each source
sample_k_small_clusters = np.random.choice(df_err_grouped_k_small.pregroup_cluster.unique(), size=10, replace=False)
sample_g_small_clusters = np.random.choice(df_err_grouped_g_small.pregroup_cluster.unique(), size=10, replace=False)

In [48]:
# randomly select min(5,cluster size) crashes from each cluster
# Kaggle
for j in range(len(sample_k_small_clusters)):
    skc = sample_k_small_clusters[j]
    print('\033[1m'+"+++++{}/{}: Error examples for Cluster {} in Kaggle error notebooks.+++++\n".format(j+1, 10, skc)+'\033[0m')
    df_target = df_err_grouped_k_small[df_err_grouped_k_small.pregroup_cluster==skc]
    n = min(5, len(df_target))
    tmp = df_target.sample(n=n, random_state=30)
    for i in range(len(tmp)):
        print("-----Error example {}/{} with eid {}.-----\n".format(i+1, n, tmp.iloc[i].eid))
        util.print_traceback(tmp.iloc[i].traceback)
        print("---------------------------------------------------------------------------\n")
    print("+++++Error examples for Cluster {} in Kaggle error notebooks end.+++++\n".format(skc))

+++++1/10: Error examples for Cluster 6376 in Kaggle error notebooks.+++++

-----Error example 1/1 with eid 454f3361-031b-373d-8f77-b40c3b5dc05f.-----

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Cell In[23], line 3
      1 lst = []
      2 for l in image_df['Label'].unique():
----> 3     lst.append(image_df[image_df['Label'] == l] .sample(100, random_state = 0))
      4 # # Concatenate the DataFrames
      5 image_df = pd.concat(lst)

File /opt/conda/lib/python3.10/site-packages/pandas/core/generic.py:5858, in NDFrame.sample(self, n, frac, replace, weights, random_state, axis, ignore_index)
   5855 if weights is not None:
   5856     weights = sample.preprocess_weights(self, weights, axis)
-> 5858 sampled_indices = sample.sample(obj_len, size, replace, weights, rs)
   5859 result = self.take(sampled_indices, axis=axis)
   5861 if ignore_index:

File /opt/conda/lib/python3.10/sit

In [49]:
# GitHub
for j in range(len(sample_g_small_clusters)):
    sgc = sample_g_small_clusters[j]
    print('\033[1m'+"+++++{}/{}: Error examples for Cluster {} in GitHub error notebooks.+++++\n".format(j+1, 10, sgc)+'\033[0m')
    df_target = df_err_grouped_g_small[df_err_grouped_g_small.pregroup_cluster==sgc]
    n = min(5, len(df_target))
    tmp = df_target.sample(n=n, random_state=30)
    for i in range(len(tmp)):
        print("-----Error example {}/{} with eid {}.-----\n".format(i+1, n, tmp.iloc[i].eid))
        util.print_traceback(tmp.iloc[i].traceback)
        print("---------------------------------------------------------------------------\n")
    print("+++++Error examples for Cluster {} in GitHub error notebooks end.+++++\n".format(sgc))

+++++1/10: Error examples for Cluster 6296 in GitHub error notebooks.+++++

-----Error example 1/1 with eid b58c4060-d217-3b5c-bfed-62b8486c11d4.-----

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<ipython-input-6-fabd3c12b267> in <module>
      3     pred_var='x',
      4     truth_var='y',
----> 5     plotvars=("precision_misspelled", "recall"),
      6 )

~/opt/anaconda3/envs/ai_academy_3_7/lib/python3.7/site-packages/wvpy/util.py in threshold_plot(d, pred_var, truth_var, truth_target, threshold_range, plotvars, title)
    365                          + str(prt_frame.columns)
    366                          + ", " + str(bad_plot_vars)
--> 367                          + " unexpected.")
    368 
    369     selector = (threshold_range[0] <= prt_frame.threshold) & (

ValueError: allowed plotting variables are: Index(['threshold', 'count', 'fraction', 'precision', 'true_positive_r

Potentially interesting clusters:

Kaggle:

        ○ Cluster 7348 (size 4): "ValueError: Cannot assign value to variable ' embedding_2/embeddings:0': Shape mismatch.The variable shape (514157, 300), and the assigned value shape (514157,) are incompatible." shape mismatch during model construction
        ○ Cluster 7274 (size 2): "InternalError: RET_CHECK failure (tensorflow/core/tpu/graph_rewrite/distributed_tpu_rewrite_pass.cc:2008) arg_shape.handle_type != DT_INVALID  input edge: [id=881 sequential_keras_layer_2728:0 -> cluster_train_function:63] [Op:__inference_train_function_3219]" due to (probably) improper training dataset size
        ○ Cluster 7345 (size 4): "RuntimeError: stack expects each tensor to be equal size, but got [19] at entry 0 and [21] at entry 1"
	
GitHub:

        ○ Cluster 251 (size 13): "RuntimeError: Calculated padded input size per channel: (1 x 1). Kernel size: (3 x 3). Kernel size can't be greater than actual input size" (torch)
        ○ Cluster 7037 (size 25): "ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 2 dimension(s)" (numpy)

In [84]:
# check if any of them are included in the samples
df_mlerr_labels = pd.read_excel(config.path_default.joinpath('Manual_labeing/df_err_processed_pregroup_sampled4_resampled7.xlsx'),
                                sheet_name = "Del-All",
                                keep_default_na=False)

C:\Users\yirwa29\AppData\Local\anaconda3\envs\chatgpt\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [85]:
for i in [7348,7274,7345,251,7037]:
    print("{} cluster size".format(i))
    print("    Kaggle: {}".format(sum(df_err_grouped_k.pregroup_cluster==i)))
    print("    GitHub: {}".format(sum(df_err_grouped_g.pregroup_cluster==i)))
    print("    If being sampled: {}".format(sum(df_mlerr_labels.pregroup_cluster==i)>0))
#     print("\n")

7348 cluster size
    Kaggle: 4
    GitHub: 1
    If being sampled: False
7274 cluster size
    Kaggle: 2
    GitHub: 0
    If being sampled: False
7345 cluster size
    Kaggle: 4
    GitHub: 12
    If being sampled: False
251 cluster size
    Kaggle: 0
    GitHub: 13
    If being sampled: False
7037 cluster size
    Kaggle: 1
    GitHub: 25
    If being sampled: False
